In [24]:
# MySQL - 조회하는 방법
import json
with open('C:\\Workspace\\python\\Data_Science\\dataA\\mysql.json') as fp:
    config_str = fp.read()
config = json.loads(config_str)

import pymysql
conn = pymysql.connect(
 host = config['host'],
 user = config['user'],
 password = config['password'],
 database = config['database'],
 port = config['port']
) 

In [ ]:
sql_select = '''
    SELECT uid, uname, email,
	    DATE_FORMAT(reg_date, '%Y-%m-%d %H:%i')AS reg_date
        FROM users WHERE is_deleted =0;
'''
cur = conn.cursor()
cur.execute(sql_select)
row = cur.fetchone()
row

In [26]:
cur = conn.cursor()
cur.execute(sql_select)
rows = cur.fetchall()
rows

(('admin', '관리자', 'admin@mc.com', '2021-08-25 13:30'),
 ('djy', '대조영', 'djy@mc.com', '2021-08-25 13:45'),
 ('eskim', '김은숙', 'eskim@mc.com', '2021-08-25 13:37'),
 ('wjlee', '이우정', 'wjlee@mc.com', '2021-08-25 13:37'))

In [27]:
cur = conn.cursor()
cur.execute(sql_select)
for row in cur:
    print(row)

('admin', '관리자', 'admin@mc.com', '2021-08-25 13:30')
('djy', '대조영', 'djy@mc.com', '2021-08-25 13:45')
('eskim', '김은숙', 'eskim@mc.com', '2021-08-25 13:37')
('wjlee', '이우정', 'wjlee@mc.com', '2021-08-25 13:37')


In [28]:
uid = 'eskim'
sql_search = """
    SELECT uid, uname, email,
	    DATE_FORMAT(reg_date, '%%Y-%%m-%%d %%H:%%i')AS reg_date
        FROM users WHERE is_deleted =0 AND uid=%s;
"""

In [29]:
cur = conn.cursor()
cur.execute(sql_search, (uid,))
result = cur.fetchone()
result

('eskim', '김은숙', 'eskim@mc.com', '2021-08-25 13:37')

In [30]:
if result:
    print(result)
else:
    print(f'uid={uid}인 사용자는 없음')

('eskim', '김은숙', 'eskim@mc.com', '2021-08-25 13:37')


In [31]:
cur.close()
conn.close()

In [32]:
# 패스워드 암호화
import hashlib
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest() # 평문을 비문으로 변경
# Base64로 인코딩
import base64
print(base64.b64encode(pwd_sha256.digest())) # 해당값은 바이트 값이기에 코드로 바로 사용하는 것은 불가능
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('UTF-8')
print(hashed_pwd)
len(hashed_pwd)

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='
A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ=


44

In [33]:
# 패스워드를 생성하는 함수
import hashlib, base64
def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('UTF-8')
    return hashed_pwd

In [34]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

In [35]:
import json
with open('C:\\Workspace\\python\\Data_Science\\dataA\\mysql.json') as fp:
    config_str = fp.read()
config = json.loads(config_str)

import pymysql
conn = pymysql.connect(
 host = config['host'],
 user = config['user'],
 password = config['password'],
 database = config['database'],
 port = config['port']
) 
# users table에 패스워드 넣기
# uid + 21
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',), ('djy',), ('eskim',), ('wjlee',))

In [36]:
sql_email = 'update users set email=%s where uid=%s;'
for line in results:
    uid = line[0]
    email = uid + '@mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()

In [37]:
sql_update = 'update users set pwd=%s where uid=%s;'
for line in results:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update, (pwd, uid))
conn.commit()

In [38]:
# 로그인
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'
# case 1: uid가 없는 경우
uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone() # result가 null
print(result)

None


In [39]:
# case 2 : 비밀번호가 틀린 경우
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')
# case 3 : OK
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Incorrect password!
Login


In [40]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Login
